# Análise de Demanda e Oferta - Microeconomia

Este notebook demonstra como usar a biblioteca de microeconomia para analisar curvas de demanda e oferta, encontrar o equilíbrio de mercado e calcular elasticidades.

In [ ]:
# Instalação das dependências (execute apenas uma vez)
%pip install numpy pandas matplotlib seaborn plotly scipy

In [ ]:
# Importações necessárias
import sys
import os

# Adicionar o diretório do projeto ao PATH
sys.path.append('/Workspace/Repos/your-repo-name')

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Importar nossa biblioteca de microeconomia
from microeconomics import (
    linear_demand, linear_supply, 
    find_equilibrium, market_analysis, plot_market,
    price_elasticity, elasticity_summary
)

## 1. Criando Curvas de Demanda e Oferta

Vamos criar curvas lineares de demanda e oferta para um produto hipotético.

In [ ]:
# Criar curva de demanda: Qd = 100 - 2*P
demand = linear_demand(intercept=100, slope=2)

# Criar curva de oferta: Qs = -20 + 3*P
supply = linear_supply(intercept=-20, slope=3)

print("Curva de Demanda: Qd = 100 - 2*P")
print("Curva de Oferta: Qs = -20 + 3*P")

## 2. Encontrando o Equilíbrio de Mercado

In [ ]:
# Encontrar o equilíbrio de mercado
eq_price, eq_quantity = find_equilibrium(demand, supply)

print(f"Preço de Equilíbrio: R$ {eq_price:.2f}")
print(f"Quantidade de Equilíbrio: {eq_quantity:.2f} unidades")

## 3. Análise Completa do Mercado

In [ ]:
# Realizar análise completa do mercado
analysis = market_analysis(demand, supply)

print("=== ANÁLISE DE MERCADO ===")
print(f"Preço de Equilíbrio: R$ {analysis['equilibrium_price']:.2f}")
print(f"Quantidade de Equilíbrio: {analysis['equilibrium_quantity']:.2f}")
print(f"Excedente do Consumidor: R$ {analysis['consumer_surplus']:.2f}")
print(f"Excedente do Produtor: R$ {analysis['producer_surplus']:.2f}")
print(f"Excedente Total: R$ {analysis['total_surplus']:.2f}")

## 4. Visualização do Mercado

In [ ]:
# Criar gráfico interativo do mercado
fig = plot_market(demand, supply, title="Análise de Mercado - Demanda vs Oferta")
fig.show()

## 5. Simulação de Dados de Mercado

In [ ]:
# Gerar dados simulados de preço e quantidade
np.random.seed(42)
prices = np.linspace(10, 40, 20) + np.random.normal(0, 1, 20)
quantities_demanded = 100 - 2*prices + np.random.normal(0, 3, 20)

# Criar DataFrame
market_data = pd.DataFrame({
    'price': prices,
    'quantity': np.maximum(0, quantities_demanded),  # Garantir quantidades não-negativas
    'income': np.random.normal(50000, 10000, 20)  # Renda simulada
})

display(market_data.head(10))

## 6. Cálculo de Elasticidades

In [ ]:
# Calcular elasticidade-preço da demanda
price_elast = price_elasticity(market_data['price'], market_data['quantity'])
avg_price_elasticity = np.mean(price_elast)

print(f"Elasticidade-Preço Média da Demanda: {avg_price_elasticity:.3f}")

if abs(avg_price_elasticity) < 1:
    print("A demanda é INELÁSTICA (variações no preço têm pouco efeito na quantidade)")
elif abs(avg_price_elasticity) > 1:
    print("A demanda é ELÁSTICA (variações no preço têm grande efeito na quantidade)")
else:
    print("A demanda tem ELASTICIDADE UNITÁRIA")

In [ ]:
# Resumo completo de elasticidades
elasticity_report = elasticity_summary(market_data, 'price', 'quantity', 'income')
display(elasticity_report)

## 7. Análise de Mudanças de Mercado

In [ ]:
# Simular um choque de demanda (aumento na demanda)
demand_shift = linear_demand(intercept=120, slope=2)  # Demanda aumentada

# Novo equilíbrio
new_eq_price, new_eq_quantity = find_equilibrium(demand_shift, supply)

print("=== APÓS CHOQUE POSITIVO DE DEMANDA ===")
print(f"Novo Preço de Equilíbrio: R$ {new_eq_price:.2f} (era R$ {eq_price:.2f})")
print(f"Nova Quantidade de Equilíbrio: {new_eq_quantity:.2f} (era {eq_quantity:.2f})")

# Visualizar ambos os cenários
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Mercado Original', 'Após Choque de Demanda')
)

# Gráfico original
fig_original = plot_market(demand, supply)
for trace in fig_original.data:
    fig.add_trace(trace, row=1, col=1)

# Gráfico após choque
fig_new = plot_market(demand_shift, supply)
for trace in fig_new.data:
    fig.add_trace(trace, row=1, col=2)

fig.update_layout(height=500, title_text="Comparação: Antes e Depois do Choque de Demanda")
fig.show()

## 8. Análise de Diferentes Tipos de Curvas

In [ ]:
# Comparar diferentes tipos de demanda
from microeconomics.demand import DemandCurve

# Demanda linear
demand_linear = linear_demand(intercept=100, slope=2)

# Demanda logarítmica
demand_log = DemandCurve('log', log_coeff=-20, intercept=100)

# Demanda potencial
demand_power = DemandCurve('power', coefficient=1000, elasticity=1.5)

# Plotar todas as curvas
prices = np.linspace(1, 30, 100)

fig = go.Figure()

# Adicionar curva linear
q_linear = demand_linear.quantity_demanded(prices)
mask_linear = q_linear >= 0
fig.add_trace(go.Scatter(
    x=q_linear[mask_linear], y=prices[mask_linear],
    mode='lines', name='Linear', line=dict(color='blue')
))

# Adicionar curva logarítmica
q_log = demand_log.quantity_demanded(prices)
mask_log = q_log >= 0
fig.add_trace(go.Scatter(
    x=q_log[mask_log], y=prices[mask_log],
    mode='lines', name='Logarítmica', line=dict(color='red')
))

# Adicionar curva potencial
q_power = demand_power.quantity_demanded(prices)
mask_power = q_power >= 0
fig.add_trace(go.Scatter(
    x=q_power[mask_power], y=prices[mask_power],
    mode='lines', name='Potencial', line=dict(color='green')
))

fig.update_layout(
    title='Comparação de Diferentes Formas Funcionais de Demanda',
    xaxis_title='Quantidade',
    yaxis_title='Preço',
    template='plotly_white'
)

fig.show()

## 9. Exportar Resultados

In [ ]:
# Criar relatório final
report = pd.DataFrame({
    'Métrica': ['Preço de Equilíbrio', 'Quantidade de Equilíbrio', 
                'Excedente do Consumidor', 'Excedente do Produtor', 'Excedente Total'],
    'Valor Original': [eq_price, eq_quantity, analysis['consumer_surplus'], 
                       analysis['producer_surplus'], analysis['total_surplus']],
    'Valor Após Choque': [new_eq_price, new_eq_quantity, 
                          market_analysis(demand_shift, supply)['consumer_surplus'],
                          market_analysis(demand_shift, supply)['producer_surplus'],
                          market_analysis(demand_shift, supply)['total_surplus']]
})

display(report)

# Salvar em arquivo CSV (opcional)
# report.to_csv('/tmp/analise_mercado.csv', index=False)
# market_data.to_csv('/tmp/dados_mercado.csv', index=False)